<a href="https://colab.research.google.com/github/LetsTrie/COVID-19-BD-Data/blob/master/test_rates/TestRateClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.linear_model import Ridge
from pandas import DataFrame
from numpy import *
import copy
import json, io
import sys
import math

In [ ]:
dataset = pd.read_csv('data.csv')
df = dataset.copy()
datasetArray = df.values
dataset.head()

,Country,Population,Density,PCI,GDP,iDensity,BPH,Test Rate
0,United Kingdom,67886004,272,39753,2698672317012,21998.393,2.54,3485
1,France,65273512,122,38605,2519883930760,32586.741,5.98,1568
2,Spain,46754783,93,34272,1602379922976,33295.631,2.97,2329
3,Germany,83783945,237,45229,3789464048405,10959.964,8.00,1885
4,Poland,37846605,124,27216,1030033201680,22286.675,6.62,935


In [ ]:
inputSet = []
testRates = []

for dataset in datasetArray:
    x = []
    for i in range(1, len(dataset)-1):
        x.append(dataset[i])
    inputSet.append(x)
    testRates.append(dataset[len(dataset) - 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(inputSet, testRates, test_size=0.2)
print(len(X_train))

12


In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(inputSet, testRates)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
bd_data = [[165379406], [1265], [4570], [2703.763], [0.8]]
print(model.predict(bd_data)[0])
bd_test_rate = (model.predict(bd_data)[0] * 100) / 100000
print(bd_test_rate)

#required rate = 2.885 %
#currrent rate = 1.518 %
#tests per thousand = 0.106, % = 0.0106
#positive per million = 11.215, % = 0.000112
#current test positivity rate = 10.6%

2885
2.885


In [36]:
main_data = pd.read_csv('owid-covid-data.csv')
main_data = main_data.fillna(method='ffill')
main_data = main_data.fillna(method='bfill')
main_data_frame = main_data.copy()
main_data_frame = main_data_frame.values

In [43]:
# 10, 27, 35, 36, 40, 47

countries = set([])
for i in range(len(main_data_frame)):
  countries.add(main_data_frame[i][2])

country_data = []

def filter(country):
  data = []
  for i in range(len(main_data_frame)):
    if main_data_frame[i][2] == country:
      row = main_data_frame[i]
      dropped_row = [row[35], row[36], row[40], row[35]*row[40], row[10], row[47], row[28]]
      data.append(dropped_row)

  return data[len(data)-1]

for country in countries:
  selected_countries = filter(country)
  country_data.append(selected_countries)

In [44]:
def form_param_array(param_no):
  dataset = []
  for i in range(len(country_data)):
     dataset.append(country_data[i][param_no])
  
  return dataset

def form_dataset(low, high, param_no):
  dataset = []
  for country in country_data:
    if low <= country[param_no] <= high:
      dataset.append([country[param_no], country[6]])
  return dataset

def mean_deviation(dataset):
  mean = np.mean(dataset)
  mean_sum = 0
  for n in dataset:
    mean_sum += abs(mean - n)

  return mean_sum/len(dataset)

In [45]:
def weighted_mean(data, bd_data):
  weights = []
  avg = 0
  print(len(data))

  if len(data) == 0 or np.isnan(data).any():
    return 0

  for i in range(len(data)):
    weights.append(1 / abs(bd_data - data[i][0]))
  _max = np.max(weights)
  norm_weights = list(map(lambda x: x/_max, weights)) 
  for i in range(len(data)):
    avg += data[i][1]*norm_weights[i]
  avg = avg / np.sum(norm_weights)

  return avg

In [50]:
bd_data = [165379406, 1265, 4570, 2740000000, 2703.763, 0.8]
allowed_std = 1
AVGs = []
# print(np.mean(form_param_array(6)))
for i in range(0, 6):
  param_array = form_param_array(i)
  mean_dev = mean_deviation(param_array)
  low = abs(bd_data[i] - allowed_std*mean_dev)
  high = bd_data[i] + allowed_std*mean_dev
  dataset = form_dataset(low, high, i)
  avg = weighted_mean(dataset, bd_data[i])
  AVGs.append(avg)

# print(AVGs)
  